In [1]:
# means that my matplotlib graphs will be included in the notebook, next to the code
%matplotlib inline

import os

import math
import astropy
import random
import numpy as np
import tables as tb
import pandas as pd
import matplotlib.pyplot as plt

from astropy.table import Table, Column, join
from astropy.coordinates import SkyCoord
from astropy.io import fits
import astropy.units as u

from hetdex_tools.get_spec import get_spectra
from hetdex_api.config import HDRconfig
from hetdex_api.detections import Detections
from hetdex_api.elixer_widget_cls import ElixerWidget

In [2]:
# not sure why the code below is here, it was in the Detections database and API notebook
# https://github.com/HETDEX/hetdex_api/blob/master/notebooks/api-notebooks/03-Detections_Database_and_API.ipynb

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

### Opens the catalogs and turns them into dataframes

I like to open both catalogs separately since they are both big (HDR3 especially!)

In [4]:
# Opening HDR3 detections catalog and converting it into a pandas DF
# HDR3 is detections HETDEX found
HDR_source_cat = fits.open('/home/jovyan/Hobby-Eberly-Telesco/hdr3/catalogs/source_catalog_3.0.1.fits', memmap = True)
HDR3_data = HDR_source_cat[1].data
HDR3_DF = pd.DataFrame(HDR3_data, columns=HDR3_data.columns.names)

In [5]:
# Opening H20 NEP catalog and converting it into a pandas DF
# H20 is stricly photometry sources
H20_NEP_catalog = fits.open('H20_NEP_VIRUS_OVERLAP_CAT_10_2021.fits', memmap = True)
H20_NEP_data = H20_NEP_catalog[1].data
H20_NEP_DF = pd.DataFrame(H20_NEP_data, columns=H20_NEP_data.columns.names)

In [6]:
# Columns we will then take from the entire data set (it was huge so we needed to determine what we wanted to look at specifically).
# As the name suggests, these are the ones that are useful to us!
useful_hdr3_cols = ['source_id', 'detectid',  'selected_det', 'ra_mean', 'dec_mean', 'fwhm', 'shotid', 'field',  'ra', 'dec', 'wave', 'wave_err', 'flux', 'flux_err', 'sn', 'sn_err', 'chi2', 'chi2_err',
'linewidth', 'linewidth_err', 'plya_classification', 'z_hetdex', 'z_hetdex_conf', 'combined_plae']

# For now, the only useful columns for us in H20 NEP is RA and DEC.
useful_h20nep_cols = ['RA_MODELING', 'DEC_MODELING', 'VALID_SOURCE_MODELING']

# From the original DFs, taking the useful columns
reduced_hdr3_df = HDR3_DF.loc[:, useful_hdr3_cols]
reduced_h20nep_df = H20_NEP_DF.loc[:, useful_h20nep_cols]

### Cleaning up the data

In [7]:
# Removing data from before 2017 because it isn't good (not useful to us)
# No need to do this for H20 NEP
removed_bad_shots_hdr3_df = reduced_hdr3_df[reduced_hdr3_df.shotid.values >= 20180000000]

### Filtering data. For HDR3 we use a signal to noise greater than 6.5 and for H20 NEP we check if the VALID_SOURCE_MODELING is true

In [8]:
# This will give high confidence detections. Something we would want to do also. What is sn threshold that Valentina's code is having trouble with.
# Reason why, we want high-confidence Lya. If we are very confident sn and another filter, then that's what we consider high-conf lya.
# Once noise and high-confidence sample. We can start exanping on valentina's code and do our own stuff
hdr3_signal_to_noise_interval = removed_bad_shots_hdr3_df[removed_bad_shots_hdr3_df['sn'] > 6.5]

# For now, no need to specify a field. But once trained, we want to run this for the NEP field!
# The VALID_SOURCE_MODELING column which just tells us that the model was able to converge and get fluxes from the source
# False valid source modeling means that the model used to measure the fluxes failed somehow so we cannot use that galaxy reliably
# We want the true ones since we know the model was able to find a galaxy and we can use that for 
# our imaging counterpart identification (ie: to use these galaxy to check if there is a galaxy at our new extraction coordinate)
h20_valid_source_check = reduced_h20nep_df[reduced_h20nep_df['VALID_SOURCE_MODELING'] == True]

## Making two skycoord objects for both catalogs
We do this instead of directly looking at the dataframes cause when we do and exract we get fibers from nearby as well. We don't want that.

In [9]:
hdr3_skycoords = SkyCoord(hdr3_signal_to_noise_interval['ra'] * u.deg, hdr3_signal_to_noise_interval['dec'] * u.deg)
h20_skycoords = SkyCoord(h20_valid_source_check['RA_MODELING'] * u.deg, h20_valid_source_check['DEC_MODELING'] * u.deg)

## Here I'll probably create a 'Noise_Sampling' function  

# QUESTION: Ask for best way to store 100 noise sources, for now just do numpy array

In [350]:
# Getting a random coordinate from the h20 catalog. I didn't get from the hdr3 catalog 
# because the hdr3 one is so much bigger, so it could happen where we dont have imaging overlap
random_coord = h20_skycoords[random.randint(0, h20_skycoords.size)]
# Creating an offseted skycoord (from random one we picked) to hopefully be a noise source
offset_skycoord = random_coord.directional_offset_by(200 * u.arcsec, 200 * u.arcsec)

In [351]:
# Creating two variables holding the separation of both catalogs, returns angles
sep_hdr3 = offset_skycoord.separation(hdr3_skycoords)
sep_h20 = offset_skycoord.separation(h20_skycoords)

In [352]:
# Checking if the separation is less than 1.2 arcseconds (arbitrary value)
check_hdr3 = (sep_hdr3 < 1.2 * u.arcsec)
check_h20 = (sep_h20 < 1.2 * u.arcsec)

In [362]:
sepboth = (offset_skycoord.separation(hdr3_skycoords), offset_skycoord.separation(h20_skycoords))

In [363]:
checksep = sepboth[0] or sepboth[1] < 1.2 * u.arcsec

In [364]:
if True in checksep:
    print('change')
else:
    print("extract")

extract


In [356]:
# If there is a true value in any of the checks, we need to find a new source because it means there was a close match
if (True in check_hdr3) or (True in check_h20):
    print('need to change source')
else:
    # we extract!!
    print('extract')
    #extraction = get_spectra(offset_skycoord)
    offset_skycoord

need to change source


In [63]:
offset_skycoord

<SkyCoord (ICRS): (ra, dec) in deg
    (270.95618973, 67.27747307)>

0 means we change offset or another random coordinate

1 great! Makes our job easier, we have noise spectrum

2 or more, what do we do? Bounce off ideas. "Weighted stacking" might be an option! However might not be ideal for comparing to real data.

In [16]:
extraction

ID,shotid,wavelength,spec,spec_err,apcor,flag,gal_flag,amp_flag,meteor_flag
,,Angstrom,1e-17 erg / (Angstrom cm2 s),1e-17 erg / (Angstrom cm2 s),,,,,
int64,int64,float64[1036],float64[1036],float64[1036],float64[1036],int64,int64,int64,int64
1,20210616021,3470.0 .. 5540.0,-1.0332901577661509 .. -0.5001153100141618,6.543976661339851 .. 1.8833140977926013,0.9179288166985348 .. 0.9323621325512689,1,1,1,1


In [17]:
extraction['wavelength'].shape

(1, 1036)

In [18]:
%matplotlib inline

In [104]:
plt.plot(extraction['wavelength'][0], extraction['spec'][0])

NameError: name 'extraction' is not defined

## Note: 
Ran into this bug where if I tried to combine my checks into one tuple or variable everything would pass and I just added all sources.

In [373]:
# Input: takes as input a SkyCoord object (ra and dec coordinates)
#        an offset to move away from whatever random object's coordinates we get 
#        a separation_check which will be a number we use to see if an object is far enough away
#        and a num_sources which is just how many sources we want to end up with in our noise sample
#        any additional skycoord objects needed, in future could implement this to take an array of skycoord objects
#        just in case if there's multiple skycoord objects
#
# Default: by default we take the h20_skycoords object which uses the h20 photometry catalog to make a SkyCoord
#          (probably a good idea to clean up and filter data before puttng into function)
#          an offset of 200 arcseconds (using Skycoord separation method)
#          a separation check of 1.2 arcseconds
#          a number of 100 sources outputted
#          an additional skycoord for the HDR3 catalog
#
# Output: returns a numpy array of noise sources
def Noise_Sampling(num_sources,
    skycoord = h20_skycoords, 
    offset = 200 * u.arcsec, 
    separation_check = 1.2 * u.arcsec,
    additional_skycoord = hdr3_skycoords):
    
    # variable to count how many noise sources we have
    source_count = 0
    
    # Creating array to hold all sources to be returned later
    source_list = []
    
    while source_count < num_sources:

        #pick a random coordinate from the skycoord object
        random_coord = skycoord[random.randint(0, skycoord.size)]

        # Creating an offseted skycoord (from random one we picked) to hopefully be a noise source
        offset_skycoord = random_coord.directional_offset_by(200 * u.arcsec, 200 * u.arcsec)

        # Creating a tuple holding the separation of both catalogs, returns angles
        # Could be implemented to take an arbitrary amount of skycoord objects
        skycoords_separation = (offset_skycoord.separation(skycoord), offset_skycoord.separation(additional_skycoord))
        # Creating two variables holding the separation of both catalogs, returns angles
        sep_hdr3 = offset_skycoord.separation(skycoord)
        sep_h20 = offset_skycoord.separation(additional_skycoord)

        # Checking if the separation is less than 1.2 arcseconds (arbitrary value)
        check_separation = skycoords_separation[0] or skycoords_separation[1] < 1.2 * u.arcsec
        # Checking if the separation is less than 1.2 arcseconds (arbitrary value)
        check_hdr3 = (sep_hdr3 < 1.2 * u.arcsec)
        check_h20 = (sep_h20 < 1.2 * u.arcsec)
        
        # If there is a true value in any of the checks, we need to find a new source because it means there was a close match
        if (True in check_hdr3) or (True in check_h20):
            print('need to change source')
        else:
            # we extract!!
            print('extract')
            #extraction = get_spectra(offset_skycoord)
            #offset_skycoord

        # If there is a true value in any of the checks, we need to find a new source because it means there was a close match

        if True in check_separation:
            print('FAKEEEEE need to change source')
            continue
        else:
            #print(check_separation)
            # We store the noise source in an array
            #print(source_list)
            #print(offset_skycoord)
            #source_list.append(offset_skycoord)
            #print(len(source_list))
            #print(source_list)
            #extraction = get_spectra(offset_skycoord)
            source_count += 1
        
    return np.array(source_list)

In [372]:
test = Noise_Sampling(100)

extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
need to change source
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
need to change source
extract
extract
need to change source
need to change source
extract
extract
extract
need to change source
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
need to change source
extract
need to change source
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
need to change source
need to change source
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
extract
need to change source
extract
extract
extract
extract
extract
extract
extract
extract
extract
need to change source
extract
need to change source
extract
need to change source
extract
extract
extract
extract
extract


In [277]:
test

array([<SkyCoord (ICRS): (ra, dec) in deg
           (269.06752659, 67.06709466)>  ,
       <SkyCoord (ICRS): (ra, dec) in deg
           (271.44669137, 67.34896796)>  ,
       <SkyCoord (ICRS): (ra, dec) in deg
           (270.91265439, 67.70498659)>  ,
       <SkyCoord (ICRS): (ra, dec) in deg
           (271.12798537, 67.30861347)>  ,
       <SkyCoord (ICRS): (ra, dec) in deg
           (269.96438372, 67.21308457)>  ,
       <SkyCoord (ICRS): (ra, dec) in deg
           (269.23321892, 67.0463074)>   ,
       <SkyCoord (ICRS): (ra, dec) in deg
           (268.45116452, 66.94946783)>  ,
       <SkyCoord (ICRS): (ra, dec) in deg
           (270.76303126, 67.18333442)>  ,
       <SkyCoord (ICRS): (ra, dec) in deg
           (267.88347933, 67.48395853)>  ,
       <SkyCoord (ICRS): (ra, dec) in deg
           (271.01939117, 67.21702905)>  ,
       <SkyCoord (ICRS): (ra, dec) in deg
           (270.97428227, 67.19203271)>  ,
       <SkyCoord (ICRS): (ra, dec) in deg
           (268.2166568

In [52]:
# counter that will
random_source_count = 0
while random_source_count < 100:
    random_coord = h20_skycoords[random.randint(0, h20_skycoords.size)]
    offset_skycoord = random_coord.directional_offset_by(200 * u.arcsec, 200 * u.arcsec)
    sep_hdr3 = offset_skycoord.separation(hdr3_skycoord)
    sep_h20 = offset_skycoord.separation(h20_skycoord)
    
    random_source_count +=1

NameError: name 'hdr3_skycoord' is not defined

## Question: So now do I just put this in a for loop? What do I do now that I have one noise spectrum?

(Function) While loop over 100 random coordinates. Basically repeat this 100 times and then extract. 

Think about how to save noise spectra. A lot of options (numpy, pandas (empty DF, append spec as a column, and transpose so its all rows)).

Once spectrum, check length of table. If 0 no fiber coverage, then need to change things around to get a spectrum. No spectrum means wasn't able to get fiber.

## NOTES

We want no source. This catalog has HETDEX detections one. 

    Want to make sure:
        1.No hetdection detec
        
        2. No imaging counterpart. Do some cross-matching. Gives us a 0 and THEN we extract. Want to extract in basically empty space. Start with 100. 
        
            Coordinates still. Trying to see if no match with the .fits file.

Start with detection. One approach was fits file with coordinates. 

Or 

Use this but expand upon it. Find RA and DEC of each shot. And randomly extract.

    delta ra and delta dec. Double check if is there a source there. 
    
For noise sample, no need to run through valentina's code. Only focus on High-z after filtering through Valentina's code.

Once we have noise sample.

Run through valentina's code. Hopefully it detects them all as high-z. Cause neither low-z or star.

Two skycoords. Check coordinates to see if HETDEX detection is there. Compare minimum separation. If the difference is smaller than 3 arcseconds. Then there is a source there, so do not extract there.